In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# 导入我们上次的成果，并且附加一些我们这次需要的功能
１．　获取数据 

２．　归一化

In [2]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
x_train,y_train,x_valid,y_valid = get_data()

In [4]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

# 注意：
我们对于validation set的归一化也是用train set的标准差和均值

In [5]:
ｘ_train = normalize(x_train,train_mean,train_std)
x_valid = normalize(x_valid,train_mean,train_std)

In [6]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(-6.2598e-06), tensor(1.))

In [7]:
#export
def test_near_zero(a,tol=1e-3):
    assert a.abs()<tol, f"Near zero: {a}"

In [8]:
test_near_zero(x_train.mean())
test_near_zero(x_train.std()-1)

In [9]:
n,m = x_train.shape
c = y_train.max()+1 # 0-9 
n,m,c

(50000, 784, tensor(10))

# 简单版本的ＦＣ
1. one hidden layer

２．output size = 1

３．num hidden = 50

In [10]:
nh = 50

## 何凯明init 用行的平方根来

In [11]:
w1,b1 = torch.randn(m,nh)/math.sqrt(m),torch.zeros(nh)
w2,b2 = torch.randn(nh,1)/math.sqrt(nh),torch.zeros(1)

In [12]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [13]:
x_valid.mean(),x_valid.std() #对应０，１　

(tensor(-0.0059), tensor(0.9924))

In [14]:
def lin(x,w,b): return x@w+b
def relu(x): return x.clamp_min(0) #截取0以下的部分

In [15]:
t = lin(x_valid,w1,b1)
t.mean(),t.std()

(tensor(0.0418), tensor(0.9822))

这个时候均值和标准差就不是接近０，１了，不是很好

In [17]:
t = relu(lin(x_valid,w1,b1))
t.mean(),t.std()

(tensor(0.4102), tensor(0.5776))

这个结果也不理想，因为relu是把０下面的数据都弄掉了，因此标准差和均值会变化

## kaiming init
fan_out: forward pass -m

fan_in: backward pass -nh

In [21]:
#export
from torch.nn import init

In [23]:
#w1 = torch.randn(m,nh)*math.sqrt(2/m)
w1 = torch.randn(m,nh)
init.kaiming_normal(w1,mode='fan_out')
t = relu(lin(x_valid,w1,b1))

/home/shaobowang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
w1.mean(),w1.std()

(tensor(0.0001), tensor(0.0505))

In [25]:
t.mean(),t.std()

(tensor(0.5427), tensor(0.8215))

# relu真的好吗

In [26]:
def relu(x): return x.clamp(0.)-0.5

In [27]:
w1 = torch.randn(m,nh)*math.sqrt(2./m)
t1 = relu(lin(x_valid,w1,b1))
t1.mean(),t1.std()

(tensor(0.0672), tensor(0.8432))

In [28]:
def model(xb):
    l1 = lin(xb,w1,b1)
    l2 = relu(l1)
    l3 = lin(l2,w2,b2)
    return l3

In [29]:
%timeit -n 10 _=model(x_valid)

12.9 ms ± 3.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

# 损失函数

In [32]:
model(x_valid).shape

torch.Size([10000, 1])

In [38]:
def mse(output,targ): return ((output.squeeze(-1)-targ)**2).mean()

In [34]:
y_train,y_valid = y_train.float(),y_valid.float()

In [35]:
preds = model(x_train)

In [36]:
preds.shape

torch.Size([50000, 1])

In [39]:
mse(preds,y_train)

tensor(30.0742)

# 定义梯度

In [40]:
def mse_grad(inp,targ):
    inp.g = 2.*(inp.squeeze() - targ).unsqueeze(-1)/inp.shape[0]

In [47]:
def relu_grad(inp,out):
    inp.g = (inp>0).float()*out.g

In [48]:
def lin_grad(inp,out,w,b):
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1)*out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [49]:
def forward_and_backward(inp,targ):
    #前向传播
    l1 = inp@w1+b1
    l2 = relu(l1)
    out = l2@w2+b2
    
    loss = mse(out,targ)
    
    #反向传播
    mse_grad(out,targ)
    lin_grad(l2,out,w2,b2)
    relu_grad(l1,l2)
    lin_grad(inp,l1,w1,b1)

In [50]:
forward_and_backward(x_train,y_train)

In [56]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

# 检验一下正确性

In [64]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [65]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [66]:
loss = forward(xt2,y_train)

In [67]:
loss.backward()

In [68]:
test_near(w22.grad,w2g)
test_near(b22.grad,b2g)
test_near(w12.grad,w1g)
test_near(b12.grad,b1g)
test_near(xt2.grad,ig)

# 重构模型

In [85]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [86]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [87]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [88]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [89]:
w1.g,b1.g,w2.g,b2.g = [None]*4

In [90]:
model = Model(w1,b1,w2,b2)
%time loss = model(x_train,y_train)

CPU times: user 258 ms, sys: 18.2 ms, total: 276 ms
Wall time: 54.1 ms


In [91]:
%time model.backward()

CPU times: user 3.32 s, sys: 4.1 s, total: 7.43 s
Wall time: 1.44 s


# 利用Module.forward()

In [108]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [109]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [110]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [111]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [112]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [113]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()
%time loss = model(x_train,y_train)

CPU times: user 242 ms, sys: 95.1 ms, total: 337 ms
Wall time: 78.3 ms


In [114]:
%time model.backward()

CPU times: user 459 ms, sys: 319 ms, total: 778 ms
Wall time: 171 ms


In [115]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

## 不用einsum

In [116]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [117]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [118]:
%time loss = model(x_train, y_train)

CPU times: user 259 ms, sys: 118 ms, total: 377 ms
Wall time: 93.7 ms


In [119]:
%time model.backward()

CPU times: user 417 ms, sys: 173 ms, total: 590 ms
Wall time: 125 ms


In [120]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

# nn.Linear nn.Module

In [121]:
#export
from torch import nn

In [122]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [123]:
model = Model(m, nh, 1)

In [124]:
%time loss = model(x_train, y_train)

CPU times: user 237 ms, sys: 143 ms, total: 380 ms
Wall time: 85 ms


In [125]:
%time loss.backward()

CPU times: user 374 ms, sys: 121 ms, total: 494 ms
Wall time: 116 ms


# Export

In [126]:
!./notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py
